# add c_EC as one of the inputs for diffusivity and conductivity

In [23]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;
from scipy.io import savemat,loadmat;from pybamm import constants,exp,tanh;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False

In [24]:
# define function:
def electrolyte_conductivity_Nyman2008_wEC(c_e,c_EC, T):
    x = c_EC / c_e
    coff = 1
    sigma_e = (
        (0.00+ coff/2 + coff/2 *  tanh((x-2.7705)*2)) * 
        (0.1297 * (c_e / 1000) ** 3 - 2.51 * (c_e / 1000) ** 1.5 + 3.329 * (c_e / 1000)) )
    return sigma_e
def electrolyte_diffusivity_Nyman2008_wEC(c_e, c_EC,T):
    x = c_EC / c_e
    coff = 1
    D_c_e = (
        ( 0.00+ coff/2 + coff/2 *  tanh((x-2.7705)*2) )  * 
        (8.794e-11 * (c_e / 1000) ** 2 - 3.972e-10 * (c_e / 1000) + 4.862e-10) )
    return D_c_e


In [25]:
V_max = 4.2;        V_min = 2.5;
Diff_SEI = 1.7e-20;    R_SEI = 2E5;   Bulk_Sol_Con = 4541.0;
Experiment_Long   = pybamm.Experiment( [ (
    f"Discharge at 1 C until {V_min} V", 
    f"Charge at 0.3 C until {V_max} V", 
    f"Hold at {V_max} V until C/5")    ] * 1 ) 
Experiment_s   = pybamm.Experiment( [ (
    f"Discharge at 1 C for 239 s", )    ] * 1 )   

In [26]:
BasicPath = 'D:/OneDrive - Imperial College London/SimDataSave/P3R4/'; 
Target  = 'a2_Diff_Cond_c_EC/'
if not os.path.exists(BasicPath + Target):
    os.mkdir(BasicPath + Target);

In [27]:
Model_ddiff = pybamm.lithium_ion.DFN(     
    options={"solvent diffusion": "EC wo refill"     } )
Model_sdiff = pybamm.lithium_ion.DFN(     
    options={"solvent diffusion": "none"     } )
ChemistryChen=pybamm.parameter_sets.Chen2020 
ChemistryChen["electrolyte"] = "lipf6_Nyman2008_rio";
Para_0=pybamm.ParameterValues(chemistry=ChemistryChen);


Para_0['EC Lithium ion cross diffusivity [m2.s-1]'] = 1.5e-9/1e2;
Para_0['Typical EC Lithium ion cross diffusivity [m2.s-1]'] =   1.5e-9/1e2
Para_0['EC transference number'] =    -1.4
Para_0['EC diffusivity in electrolyte [m2.s-1]'] =  5E-12;
Para_0.update(
    {'Electrolyte diffusivity [m2.s-1]':
    electrolyte_diffusivity_Nyman2008_wEC})
Para_0.update(
    {'Electrolyte conductivity [S.m-1]':
    electrolyte_conductivity_Nyman2008_wEC})
c_e_ddiff = Model_ddiff.variables["Electrolyte concentration [mol.m-3]"]
T_ddiff = Model_ddiff.variables["Cell temperature [K]"]
c_EC_ddiff = Model_ddiff.variables["EC concentration [mol.m-3]"]
Model_ddiff.variables["c(EC) over c(Li+)"] = c_EC_ddiff / c_e_ddiff
Model_ddiff.variables["Electrolyte conductivity [S.m-1]"] =electrolyte_conductivity_Nyman2008_wEC(c_e_ddiff,c_EC_ddiff, T_ddiff)
Model_ddiff.variables["Electrolyte diffusivity [m2.s-1]"] =electrolyte_diffusivity_Nyman2008_wEC(c_e_ddiff,c_EC_ddiff, T_ddiff)

using EC wo refill for Li+


In [38]:
var = pybamm.standard_spatial_vars  
var_pts = {
    var.x_n: 150,  
    var.x_s: 150,  
    var.x_p: 150,  
    var.r_n: 150,  
    var.r_p: 150,  }
Sim_Landes_ddiff    = pybamm.Simulation(
    Model_ddiff, experiment = Experiment_Long,
    parameter_values = Para_0,
    solver = pybamm.CasadiSolver(),)   
Sim_Landes_sdiff    = pybamm.Simulation(
    Model_sdiff, experiment = Experiment_Long,
    parameter_values = Para_0,
    solver = pybamm.CasadiSolver(),)  
Sol_Landes_ddiff    = Sim_Landes_ddiff.solve();
Sol_Landes_sdiff    = Sim_Landes_sdiff.solve();

psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:852: Linear solve failed
At t = 0.00100051 and h = 4.17859e-015, the corrector convergence failed repeatedly or with |h| = hmin.
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
At t = 0.00100007 and h = 1.29359e-014, the corrector convergence failed repeatedly or with |h| = hmin.
psetup failed: .

SolverError: Maximum number of decreased steps occurred at t=240.0. Try solving the model up to this time only or reducing dt_max (currently, dt_max=7.091632984777611) and/or reducing the size of the time steps or period of the experiment.

In [29]:
Model_ddiff.variables.search("potential")

Electrolyte potential
Electrolyte potential [V]
Gradient of electrolyte potential
Gradient of negative electrode potential
Gradient of negative electrolyte potential
Gradient of positive electrode potential
Gradient of positive electrolyte potential
Gradient of separator electrolyte potential
Negative current collector potential
Negative current collector potential [V]
Negative electrode lithium plating reaction overpotential
Negative electrode lithium plating reaction overpotential [V]
Negative electrode open circuit potential
Negative electrode open circuit potential [V]
Negative electrode potential
Negative electrode potential [V]
Negative electrode reaction overpotential
Negative electrode reaction overpotential [V]
Negative electrode surface potential difference
Negative electrode surface potential difference [V]
Negative electrolyte potential
Negative electrolyte potential [V]
Positive current collector potential
Positive current collector potential [V]
Positive electrode open ci

In [30]:
print(Sol_Landes_ddiff["Electrolyte conductivity [S.m-1]"].entries[:,-1] )
print(Sol_Landes_ddiff["Electrolyte diffusivity [m2.s-1]"].entries[:,-1])

[0.44572009 0.44753365 0.4512694  0.45714198 0.46546062 0.47659778
 0.49092464 0.50870787 0.52998428 0.55446178 0.58150608 0.61023397
 0.63966798 0.66887733 0.69705852 0.72355419 0.74782903 0.76937312
 0.78571562 0.62689655 0.09782264 0.08712451 0.07855803 0.07205283
 0.06722248 0.0635383  0.06052543 0.05786104 0.05537623 0.05299548
 0.05068836 0.04844286 0.04625341 0.04411648 0.04202911 0.03998859
 0.03799229 0.03603765 0.03412215 0.03224327 0.02259669 0.01307427
 0.00796612 0.00602704 0.00596573 0.0086971  0.01588449 0.03131188
 0.05039588 0.06262107 0.06710057 0.06789837 0.067116   0.06511103
 0.06109496 0.0534821  0.04194129 0.02995409 0.02137987 0.0169702 ]
[3.98251112e-11 3.96398143e-11 3.92921851e-11 3.88303353e-11
 3.83311416e-11 3.79035259e-11 3.76882248e-11 3.78496409e-11
 3.85562026e-11 3.99508577e-11 4.21214621e-11 4.50842624e-11
 4.87864959e-11 5.31222781e-11 5.79505724e-11 6.31072552e-11
 6.84085963e-11 7.36421101e-11 7.83663368e-11 6.50825637e-11
 1.04091397e-11 9.469375

In [31]:
font = {'family' : 'Times New Roman',
        #'weight' : 'bold',
        'size'   : 18}
mpl.rc('font', **font)

In [32]:
label = ["Landes_ddiff"] 
output_variables3 = [
    "Terminal voltage [V]",   
    #"Current [A]",
    "EC concentration [mol.m-3]",
    "Electrolyte concentration [mol.m-3]",
    "c(EC) over c(Li+)",
    #"Loss of capacity to SEI [A.h]",
    "Li+ flux [mol.m-2.s-1]",
    "EC flux [mol.m-2.s-1]",
    "Electrolyte conductivity [S.m-1]",
    "Electrolyte diffusivity [m2.s-1]",
]
quick_plot = pybamm.QuickPlot([
    Sol_Landes_ddiff
    ], output_variables3,label,
    variable_limits='fixed',time_unit='hours',n_rows=2,
    figsize = (16,6)) #     spatial_unit='mm',
quick_plot.dynamic_plot();
#quick_plot.create_gif(
#    number_of_images=10, duration=2,output_filename=BasicPath + Target+"fixed_add c_EC as tanh (0-1).gif")

interactive(children=(FloatSlider(value=0.0, description='t', max=0.06638888888888889, step=0.0006638888888888…

In [33]:
label = ["Sol_Landes_ddiff"] 
output_variables3 = [
    "Terminal voltage [V]",   
    ["EC flux [mol.m-2.s-1]","EC flux by diffusion [mol.m-2.s-1]",
        "EC flux by migration [mol.m-2.s-1]","EC flux by Li+ [mol.m-2.s-1]"],
    ["Li+ flux [mol.m-2.s-1]","Li+ flux by diffusion [mol.m-2.s-1]",
        "Li+ flux by migration [mol.m-2.s-1]","Li+ flux by solvent [mol.m-2.s-1]"],    
]
quick_plot = pybamm.QuickPlot([
    Sol_Landes_ddiff,
    ], output_variables3,label,
    variable_limits='fixed',time_unit='hours',n_rows=1,
    figsize = (16,6)) #     spatial_unit='mm',
quick_plot.dynamic_plot();

interactive(children=(FloatSlider(value=0.0, description='t', max=0.06638888888888889, step=0.0006638888888888…

In [34]:
label = ["Sol_Landes_ddiff"] 
output_variables3 = [
    "Terminal voltage [V]",   
    "Electrolyte potential [V]",
    "Positive electrode potential [V]",
    "Negative electrode SOC",
    "Positive electrode SOC",
    "Porosity times concentration",
    "eps_c_e rhs",
    "Minus div Li+ flux",
    "Li+ source term",
]
quick_plot = pybamm.QuickPlot([
    Sol_Landes_ddiff,
    ], output_variables3,label,
    variable_limits='fixed',time_unit='hours',n_rows=3,
    figsize = (16,12)) #     spatial_unit='mm',
quick_plot.dynamic_plot();


interactive(children=(FloatSlider(value=0.0, description='t', max=0.06638888888888889, step=0.0006638888888888…

In [35]:
label = ["Landes_ddiff"] 
output_variables3 = [
    "Terminal voltage [V]",   
    #"Current [A]",
    "EC concentration [mol.m-3]",
    "Electrolyte concentration [mol.m-3]",
    "c(EC) over c(Li+)",
    #"Loss of capacity to SEI [A.h]",
    "Li+ flux [mol.m-2.s-1]",
    "EC flux [mol.m-2.s-1]",
    "Electrolyte conductivity [S.m-1]",
    "Electrolyte diffusivity [m2.s-1]",
]
quick_plot = pybamm.QuickPlot([
    Sol_Landes_ddiff
    ], output_variables3,label,
    variable_limits='tight',time_unit='hours',n_rows=2,
    figsize = (16,6)) #     spatial_unit='mm',
quick_plot.dynamic_plot();
#quick_plot.create_gif(
#    number_of_images=10, duration=2,output_filename=BasicPath + Target+"add c_EC as tanh (0-1).gif")

interactive(children=(FloatSlider(value=0.0, description='t', max=0.06638888888888889, step=0.0006638888888888…